# Power System Load Flow Analysis
## Comparison of Gauss-Seidel and Newton-Raphson Methods

This notebook demonstrates the computation of load flow using two common iterative methods:
1. **Gauss-Seidel (GS)**
2. **Newton-Raphson (NR)**

We will explore the theory, implement the algorithms in Python, solve the IEEE 9-Bus system, and visualize the convergence characteristics.

### 1. Gauss-Seidel Method
The Gauss-Seidel method is an iterative technique for solving a set of non-linear algebraic equations. It is simple to implement but has a linear convergence rate.

For a power system, the voltage at bus $i$ is updated using:

$$ V_i^{(k+1)} = \frac{1}{Y_{ii}} \left( \frac{P_i - jQ_i}{(V_i^{(k)})^*} - \sum_{j=1, j \neq i}^{N} Y_{ij} V_j \right) $$

Where:
- $P_i, Q_i$: Scheduled active and reactive power at bus $i$.
- $Y_{ij}$: Element of the Y-bus matrix.
- $V_j$: Most recent voltage value available (updated values are used immediately).

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

def build_y_bus(num_buses, branch_data):
    """
    Constructs the Y-bus matrix from branch data.
    branch_data: list of tuples (from, to, R, X, B)
    """
    Y_bus = np.zeros((num_buses, num_buses), dtype=complex)
    for branch in branch_data:
        f, t, r, x, b = branch
        i, j = int(f) - 1, int(t) - 1
        z = complex(r, x)
        y = 1 / z
        b_shunt = complex(0, b / 2)
        
        Y_bus[i, i] += y + b_shunt
        Y_bus[j, j] += y + b_shunt
        Y_bus[i, j] -= y
        Y_bus[j, i] -= y
    return Y_bus

In [ ]:
def gauss_seidel(Y_bus, P_spec, Q_spec, V_init, bus_types, max_iter=100, tol=1e-6):
    """
    Solves Load Flow using Gauss-Seidel method.
    Returns final Voltages and list of max errors per iteration.
    """
    V = np.array(V_init, copy=True)
    num_buses = len(V)
    errors = []
    
    for it in range(max_iter):
        V_prev = np.copy(V)
        for i in range(num_buses):
            if bus_types[i] == 0: continue # Skip Slack bus
            
            sum_YV = 0
            for j in range(num_buses):
                if i != j:
                    sum_YV += Y_bus[i, j] * V[j]
            
            if bus_types[i] == 2: # PV Bus
                # Estimate Q for PV bus
                Q_calc = -np.imag(np.conj(V[i]) * (sum_YV + Y_bus[i, i] * V[i]))
                S_inj = P_spec[i] - 1j * Q_calc
            else: # PQ Bus
                S_inj = P_spec[i] - 1j * Q_spec[i]
            
            # Update V
            V_new = (1 / Y_bus[i, i]) * ((S_inj / np.conj(V[i])) - sum_YV)
            
            if bus_types[i] == 2: # Enforce Magnitude for PV Bus
                V_new = np.abs(V_init[i]) * np.exp(1j * np.angle(V_new))
            
            V[i] = V_new
            
        # Calculate max error (change in voltage)
        err = np.max(np.abs(V - V_prev))
        errors.append(err)
        
        if err < tol:
            break
            
    return V, errors

### 2. Newton-Raphson Method
The Newton-Raphson method solves the polar form of the power flow equations by linearizing them using a Jacobian matrix. It has quadratic convergence (very fast near the solution).

We solve for corrections $\Delta \theta$ and $\Delta |V|$:

$$ \begin{bmatrix} \Delta P \\ \Delta Q \end{bmatrix} = \begin{bmatrix} J_{11} & J_{12} \\ J_{21} & J_{22} \end{bmatrix} \begin{bmatrix} \Delta \theta \\ \Delta |V| \end{bmatrix} $$

The Jacobian elements are partial derivatives of P and Q with respect to angles and magnitudes.

In [ ]:
def newton_raphson(Y_bus, P_spec, Q_spec, V_init, bus_types, max_iter=20, tol=1e-6):
    """
    Solves Load Flow using Newton-Raphson method.
    Returns final Voltages and list of max mismatches per iteration.
    """
    V = np.array(V_init, copy=True)
    num_buses = len(V)
    pq_buses = np.where(bus_types == 1)[0]
    pv_buses = np.where(bus_types == 2)[0]
    non_slack = np.sort(np.concatenate((pq_buses, pv_buses)))
    errors = []
    
    for it in range(max_iter):
        # Calculate Power Mismatches
        S_calc = V * np.conj(Y_bus @ V)
        P_calc = np.real(S_calc)
        Q_calc = np.imag(S_calc)
        
        dP = P_spec[non_slack] - P_calc[non_slack]
        dQ = Q_spec[pq_buses] - Q_calc[pq_buses]
        mismatch = np.concatenate((dP, dQ))
        
        err = np.max(np.abs(mismatch))
        errors.append(err)
        if err < tol: break
            
        # Jacobian Construction
        n_ns = len(non_slack)
        n_pq = len(pq_buses)
        J11 = np.zeros((n_ns, n_ns))
        J12 = np.zeros((n_ns, n_pq))
        J21 = np.zeros((n_pq, n_ns))
        J22 = np.zeros((n_pq, n_pq))
        
        # J11, J21 (Angle derivatives)
        for r, i in enumerate(non_slack):
            for c, k in enumerate(non_slack):
                if i == k:
                    J11[r, c] = -Q_calc[i] - np.imag(Y_bus[i, i]) * np.abs(V[i])**2
                else:
                    y_ik = Y_bus[i, k]
                    delta_ik = np.angle(V[i]) - np.angle(V[k])
                    J11[r, c] = np.abs(V[i]*V[k]) * (np.real(y_ik)*np.sin(delta_ik) - np.imag(y_ik)*np.cos(delta_ik))
        
        for r, i in enumerate(pq_buses):
            for c, k in enumerate(non_slack):
                if i == k:
                    J21[r, c] = P_calc[i] - np.real(Y_bus[i, i]) * np.abs(V[i])**2
                else:
                    y_ik = Y_bus[i, k]
                    delta_ik = np.angle(V[i]) - np.angle(V[k])
                    J21[r, c] = -np.abs(V[i]*V[k]) * (np.real(y_ik)*np.cos(delta_ik) + np.imag(y_ik)*np.sin(delta_ik))
                    
        # J12, J22 (Magnitude derivatives)
        for r, i in enumerate(non_slack):
            for c, k in enumerate(pq_buses):
                if i == k:
                    J12[r, c] = P_calc[i]/np.abs(V[i]) + np.real(Y_bus[i, i])*np.abs(V[i])
                else:
                    y_ik = Y_bus[i, k]
                    delta_ik = np.angle(V[i]) - np.angle(V[k])
                    J12[r, c] = np.abs(V[i]) * (np.real(y_ik)*np.cos(delta_ik) + np.imag(y_ik)*np.sin(delta_ik))
                    
        for r, i in enumerate(pq_buses):
            for c, k in enumerate(pq_buses):
                if i == k:
                    J22[r, c] = Q_calc[i]/np.abs(V[i]) - np.imag(Y_bus[i, i])*np.abs(V[i])
                else:
                    y_ik = Y_bus[i, k]
                    delta_ik = np.angle(V[i]) - np.angle(V[k])
                    J22[r, c] = np.abs(V[i]) * (np.real(y_ik)*np.sin(delta_ik) - np.imag(y_ik)*np.cos(delta_ik))
        
        # Solve
        J = np.block([[J11, J12], [J21, J22]])
        dx = np.linalg.solve(J, mismatch)
        
        # Update
        V_ang = np.angle(V)
        V_mag = np.abs(V)
        V_ang[non_slack] += dx[:n_ns]
        V_mag[pq_buses] += dx[n_ns:]
        V = V_mag * np.exp(1j * V_ang)
        
    return V, errors

### 3. Example: IEEE 9-Bus System
We will now set up the data for the standard IEEE 9-bus test system and run both methods.

In [ ]:
# IEEE 9-Bus Data
num_buses = 9
bus_types = np.array([0, 2, 2, 1, 1, 1, 1, 1, 1]) # 0:Slack, 1:PQ, 2:PV
P_spec = np.zeros(num_buses)
Q_spec = np.zeros(num_buses)

# Generators (Generation is positive)
P_spec[1] = 1.63; P_spec[2] = 0.85
# Loads (Load is negative)
P_spec[4] = -1.25; Q_spec[4] = -0.50
P_spec[5] = -0.90; Q_spec[5] = -0.30
P_spec[7] = -1.00; Q_spec[7] = -0.35

# Initial Voltages (Flat Start)
V_init = np.ones(num_buses, dtype=complex)
V_init[0] = 1.04 + 0j
V_init[1] = 1.025 + 0j
V_init[2] = 1.025 + 0j

# Branch Data: (From, To, R, X, B)
branch_data = [
    (4, 5, 0.0100, 0.0850, 0.1760), (4, 6, 0.0170, 0.0920, 0.1580),
    (5, 7, 0.0320, 0.1610, 0.3060), (6, 9, 0.0390, 0.1700, 0.3580),
    (7, 8, 0.0085, 0.0720, 0.1490), (8, 9, 0.0119, 0.1008, 0.2090),
    (1, 4, 0.0, 0.0576, 0.0), (2, 7, 0.0, 0.0625, 0.0), (3, 9, 0.0, 0.0586, 0.0)
]

Y_bus = build_y_bus(num_buses, branch_data)

In [ ]:
# Run Simulations
V_gs, err_gs = gauss_seidel(Y_bus, P_spec, Q_spec, V_init, bus_types)
V_nr, err_nr = newton_raphson(Y_bus, P_spec, Q_spec, V_init, bus_types)

# Plot Convergence
plt.figure(figsize=(10, 6))
plt.plot(err_gs, label='Gauss-Seidel', marker='o')
plt.plot(err_nr, label='Newton-Raphson', marker='x')
plt.yscale('log')
plt.xlabel('Iteration')
plt.ylabel('Max Error (log scale)')
plt.title('Convergence Comparison: GS vs NR')
plt.grid(True, which="both", ls="-")
plt.legend()
plt.show()

print(f"GS Converged in {len(err_gs)} iterations.")
print(f"NR Converged in {len(err_nr)} iterations.")

### 4. Divergence Analysis
Load flow methods can diverge if:
1. The system is heavily loaded (near voltage collapse).
2. Initial guess is too far from the solution.

Below we simulate a divergence scenario by drastically increasing the load at Bus 5.

In [ ]:
# Increase load at Bus 5 to provoke stress/divergence
P_spec_stress = P_spec.copy()
P_spec_stress[4] = -5.0 # Very high load (was -1.25)

print("Running NR on stressed system...")
V_nr_div, err_nr_div = newton_raphson(Y_bus, P_spec_stress, Q_spec, V_init, bus_types, max_iter=50)

plt.figure(figsize=(10, 6))
plt.plot(err_nr_div, label='NR (Stressed)', color='red')
plt.yscale('log')
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title('Newton-Raphson Behavior under Stress')
plt.grid(True)
plt.legend()
plt.show()